In [1]:
import time
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
import os
from openpyxl import Workbook

print("import lib oke!")

import lib oke!


In [2]:
driver_path = Path("msedgedriver.exe").resolve()
if not driver_path.is_file():
    raise FileNotFoundError(f"Không tìm thấy msedgedriver tại đường dẫn: {driver_path}")
edge_service = Service(str(driver_path)) 
driver = webdriver.Edge(service=edge_service)
print(edge_service)

In [3]:
url = "https://batdongsan.com.vn/du-an-can-ho-chung-cu-tp-hcm"
driver.get(url)

wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".re__project-main-container.re__clearfix")))
print(element)

<selenium.webdriver.remote.webelement.WebElement (session="2a6ff45dce9f72c65190241881291b6a", element="f.F323ECF1D3C12B66437380BC352F84FA.d.232B7EB25F1A306BA4EDCBB52DBEA53B.e.64")>


In [4]:
def get_element_text(element, css_selector, default="N/A"):
    try:
        return element.find_element(By.CSS_SELECTOR, css_selector).text.strip()
    except:
        return default

def save_content(write_header=False):
    project_list = []
    projects = driver.find_elements(By.CSS_SELECTOR, "div.re__prj-card-info-content")
    
    for project in projects:
        name = get_element_text(project, "h3.re__prj-card-title")
        area = get_element_text(project, "span.re__prj-card-config-value")
        apartment = get_element_text(project, "span.re__prj-card-config-value span")
        location = get_element_text(project, "div.re__prj-card-location")
        description = get_element_text(project, "div.re__prj-card-summary")
        company = get_element_text(project, "div.re__prj-card-contact-avatar.re__avatar-full-max-width span.re__span-full-max-width")

        project_list.append({
            "Tên dự án": name,
            "Diện tích": area,
            "Số căn hộ": apartment,
            "Vị trí": location,
            "Mô tả": description,
            "Tên công ty": company
        })

    df = pd.DataFrame(project_list)
    print(df)
    # Check if the file exists
    file_exists = os.path.isfile("projectExcel.xlsx")
    
    # Create the file if it does not exist
    if not file_exists:
        wb = Workbook()
        wb.save("projectExcel.xlsx")

    with pd.ExcelWriter("projectExcel.xlsx", mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
        df.to_excel(writer, index=False, header=write_header, startrow=writer.sheets['Sheet1'].max_row if 'Sheet1' in writer.sheets else 0)

    with open("projectData.json", "a", encoding="utf-8") as jsonfile:
        json.dump(project_list, jsonfile, ensure_ascii=False, indent=4)

In [5]:
# Find the last page number
pagination_group = driver.find_element(By.CLASS_NAME, "re__pagination-group")
pagination_numbers = pagination_group.find_elements(By.TAG_NAME, "a")
pid_value = pagination_numbers[-2].get_attribute("pid")
print(pid_value)
if pid_value is not None:
    last_page_number = int(pid_value)
else:
    last_page_number = 0  # or handle the error as needed


73


In [ ]:
for page_number in range(1, last_page_number):
    # Save the current page content
    save_content(write_header=(page_number == 1))
    
    # Check if there is a next page
    try:
        next_page = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".re__pagination-icon .re__icon-chevron-right"))
        )
        next_page.click()
        print(f"Page {page_number}")
        time.sleep(3)  # thời gian load page tiếp theo
    except Exception as e:
        print(f"Error on page {page_number}: {e}")
        break

                            Tên dự án Diện tích Số căn hộ  \
0  Diamond Alnata Plus - Celadon City   2,39 ha       546   
1              The Galleria Residence    1,5 ha       456   
2                 Ruby - Celadon City    4,2 ha     1.488   
3              Emerald - Celadon City   4,71 ha     2.122   
4                      Diamond Alnata     82 ha     1.344   
5                   Diamond Brilliant    5,5 ha       N/A   
6                        Terra Luxury    2,6 ha       342   
7                  The Siamese Towers  6.000 m²     2.358   
8  The Opus One - Vinhomes Grand Park    2,3 ha     2.000   
9                    Chung cư Gia Phú       389       389   

                                              Vị trí  \
0  Đường N1, Phường Sơn Kỳ, Quận Tân Phú, Hồ Chí ...   
1  Lô 1-16 khu chức năng số 01, Khu đô thị Thủ Th...   
2  Đường N1, Phường Sơn Kỳ, Quận Tân Phú, Hồ Chí ...   
3  Đường N4, Phường Sơn Kỳ, Quận Tân Phú, Hồ Chí ...   
4         , Phường Sơn Kỳ, Quận Tân Phú, Hồ Chí 